# Document Question Answering with local persistence

An example of using Chroma DB and LangChain to do question answering over documents, with a locally persisted database. 
You can store embeddings and documents, then use them again later.

In [21]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import VectorDBQA, RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
import glob
import codecs

In [22]:
from environs import Env

env = Env()
env.read_env(".env")  # read .env file, if it exists
# required variables
OPENAI_API_KEY = env("OPENAI_API_KEY") 

In [23]:
# first let's encode the files as utf-8
chat_logs = glob.glob("/Users/jgourneau/Downloads/hplus_2023/*.txt")

def save_as_utf8(file_list):
    for file_name in file_list:
        with codecs.open(file_name, 'r', encoding='ISO-8859-1') as f:
            contents = f.read()
        with codecs.open(file_name, 'w', encoding='utf-8') as f:
            f.write(contents)

save_as_utf8(chat_logs)


## Load and process documents

Load documents to do question answering over. If you want to do this over your documents, this is the section you should replace.

Next we split documents into small chunks. This is so we can find the most relevant chunks for a query and pass only those into the LLM.

In [28]:
loader = DirectoryLoader('/Users/jgourneau/Downloads/hplus_2023/', glob="2023*.txt", loader_cls=TextLoader)
documents = loader.load()


In [29]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [30]:
len(documents)

98

## Initialize PeristedChromaDB

Create embeddings for each chunk and insert into the Chroma vector database. The `persist_directory` argument tells ChromaDB where to store the database when it's persisted. 

In [31]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'hplus_2023_1'
embedding = HuggingFaceEmbeddings()

In [32]:
vectordb = Chroma.from_documents(documents=texts, embedding=embedding, persist_directory=persist_directory)
#will take a day

Using embedded DuckDB with persistence: data will be stored in: hplus_2023_1


In [34]:
#locals()

## Persist the Database
In a notebook, we should call `persist()` to ensure the embeddings are written to disk.
This isn't necessary in a script - the database will be automatically persisted when the client object is destroyed.

In [35]:
vectordb.persist()
vectordb = None

## Load the Database from disk, and create the chain
Be sure to pass the same `persist_directory` and `embedding_function` as you did when you instantiated the database. Initialize the chain we will use for question answering.

In [36]:
# Now we can load the persisted database from disk, and use it as normal. 
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

qa = VectorDBQA.from_chain_type(llm=OpenAI(model_name="gpt-4"), chain_type="stuff", vectorstore=vectordb)
#qa = RetrievalQA.from_chain_type(llm=OpenAI(model_name="gpt-4"), chain_type="stuff", vectorstore=vectordb)



Using embedded DuckDB with persistence: data will be stored in: hplus_2023_1
/Users/jgourneau/miniconda3/envs/chain/lib/python3.11/site-packages/langchain/llms/openai.py:169: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/jgourneau/miniconda3/envs/chain/lib/python3.11/site-packages/langchain/llms/openai.py:623: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/jgourneau/miniconda3/envs/chain/lib/python3.11/site-packages/langchain/chains/retrieval_qa/base.py:183: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


## Ask questions!

Now we can use the chain to ask questions!

In [37]:
query = " what is llama.cpp  "
qa.run(query)

"llama.cpp is a software on GitHub that allows users to work with and run OpenAI's language models, such as the Llama and Alpaca models. It provides functionalities like interactive mode and can be used to convert files or interact with these models in various ways."

In [39]:
query = "   "
qa.run(query)

"I don't know."

## Cleanup

When you're done with the database, you can delete it from disk. You can delete the specific collection you're working with (if you have several), or delete the entire database by nuking the persistence directory.

In [ ]:
# To cleanup, you can delete the collection
#vectordb.delete_collection()
#vectordb.persist()
